# Download ERA5-LAND hourly temperatures and generate daily summary

# TODO: ERA5-Land for whole globe is something like 50GB per year: need different method to do daily summaries because 'naive' open+save causes Xarray to try to allocate RAM for the whole dataset. Might be possible to force it to avoid doing this e.g. map blocks, also need incremental write. Or just loop manually and save to netcdf instead of dicking about with xarray. Other option to do it on HPC cluster and just request really big RAM. 

# First Step is to download by Month instead of one big file. 

## Method

- Download hourly data for one whole year from ERA5 at 0.5deg resolution as GRIB file (in theory should be faster/smaller than netCDF and we will discard it anyway)
- Calculate minimum daily temperature for each day and save
- Delete the input GRIB to avoid acumulating terabytes of data!

In [2]:
from collections import namedtuple
from pathlib import Path

import numpy as np
import xarray as xr
import cdsapi

In [3]:
DATA_SRC = Path('~/Shared/Data/').expanduser()

In [4]:
SUBDAILY_TEMPERATURES_FOLDER = DATA_SRC / 'weather' / 'era5_land' / 'hourly_temperature_2m'
SUBDAILY_TEMPERATURES_FOLDER.mkdir(exist_ok=True)

TEMPERATURE_SUMMARY_FOLDER = DATA_SRC / 'weather' / 'era5_land' / 'era5_land_daily_summary'
TEMPERATURE_SUMMARY_FOLDER.mkdir(exist_ok=True)

In [5]:
MAX_YEAR = 2022

In [7]:
assert SUBDAILY_TEMPERATURES_FOLDER.is_dir()
assert TEMPERATURE_SUMMARY_FOLDER.is_dir()

In [1]:
# def retreive_year(out_file, year):
#     c = cdsapi.Client()
#     c.retrieve(
#         'reanalysis-era5-land',
#         {
#             'variable':'2m_temperature',
#             'year': year,
#             'month':[
#                 '01','02','03',
#                 '04','05','06',
#                 '07','08','09',
#                 '10','11','12'
#             ],
#             'day':[
#                 '01','02','03',
#                 '04','05','06',
#                 '07','08','09',
#                 '10','11','12',
#                 '13','14','15',
#                 '16','17','18',
#                 '19','20','21',
#                 '22','23','24',
#                 '25','26','27',
#                 '28','29','30',
#                 '31'
#             ],
#             'time':[
#                 '00:00','01:00','02:00',
#                 '03:00','04:00','05:00',
#                 '06:00','07:00','08:00',
#                 '09:00','10:00','11:00',
#                 '12:00','13:00','14:00',
#                 '15:00','16:00','17:00',
#                 '18:00','19:00','20:00',
#                 '21:00','22:00','23:00'
#             ],
#             'format':'grib'
#         },
#         str(out_file))
    
    
# def generate_daily_summary(source_file):
#     # resample and save a single year.
#     # NOTE: CANNOT load at first because ERA5 LAND is 45GB per year!!
#     daily = xr.open_dataset(source_file, engine='cfgrib')
#     daily = daily.resample(time='1D')
#     tmin = daily.min()
#     tmax = daily.max()
#     tmean = daily.mean()

#     tmin = tmin.rename({'t2m': 't_min'})
#     tmax = tmax.rename({'t2m': 't_max'})
#     tmean = tmean.rename({'t2m': 't_mean'})
#     daily_summary = xr.merge([tmin, tmax, tmean])
#     return daily_summary


# def download_and_summarise_year(year, overwrite=False):
#     '''Download the GRIB file for hourly temperatures for one year
#     Generate a summary file with temperature min, max, mean
    
#     TODO: later, probably can delete the hourly data file after summarising it
#     '''
#     year = str(year)
#     out_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}_temperature.grib'
#     summary_file = TEMPERATURE_SUMMARY_FOLDER / f'{year}_temperature_summary.nc'
    
#     if overwrite is True or out_file.exists() is False:
#         retreive_year(out_file, year)
#     else:
#         print(f'Skip {out_file}, already exists.')
    
#     if overwrite is True or summary_file.exists() is False:
#         daily_summary = generate_daily_summary(out_file)
        
#         # Encode using rounding to nearest 0.01K to save space, looses a small amount of precision
#         # but should be ok since it not used for detailed simulations...
#         daily_summary.to_netcdf(summary_file,
#                             encoding={
#                                 't_min': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
#                                 't_max': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999},
#                                 't_mean': {'dtype': 'int16', 'scale_factor': 0.01, '_FillValue': -9999}
#                             })
                           
#     else:
#         print(f'Skip {summary_file}, already exists.')

In [8]:
# EXAMPLE code for triggering many requests.
# joblib.Parallel(n_jobs=10, verbose=10)(
#     joblib.delayed(download_and_summarise_year)(year) for year in range(1980, 2020))

# DOWNLOAD PER MONTH IS TOO BIG!!
Did this until 2021 don-t know where lost it...

In [7]:
def retreive_month(out_file, year, month):
    if out_file.exists():
        print('file exists, skipping')
        return
    
    c = cdsapi.Client()
    c.retrieve(
        'reanalysis-era5-land',
        {
            'variable':'2m_temperature',
            'year': year,
            'month':[str(month).zfill(2)],
            'day':[
                '01','02','03',
                '04','05','06',
                '07','08','09',
                '10','11','12',
                '13','14','15',
                '16','17','18',
                '19','20','21',
                '22','23','24',
                '25','26','27',
                '28','29','30',
                '31'
            ],
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ],
            'format':'grib'
        },
        str(out_file))
    


In [17]:
retreive_month(out_file, year, month)

2023-02-16 11:36:03,080 INFO Welcome to the CDS
2023-02-16 11:36:03,081 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2023-02-16 11:36:03,194 INFO Request is queued
2023-02-16 11:36:05,734 INFO Request is running
2023-02-16 11:37:18,316 INFO Request is completed
2023-02-16 11:37:18,317 INFO Downloading https://download-0019.copernicus-climate.eu/cache-compute-0019/cache/data1/adaptor.mars.internal-1676543765.6679852-19764-1-aa9a2848-daa3-4821-9653-f5c71d78f4f5.grib to /home/jonathanchambers/Shared/Data/weather/era5_land/hourly_temperature_2m/2022-01_temperature_2m.nc (3.6G)
2023-02-16 11:40:17,934 INFO Download rate 20.7M/s  


# Generate daily summaries

In [11]:
SUBDAILY_TEMPERATURES_FOLDER = Path('/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_nc')

In [14]:
# Open a reference dataset
ref = xr.open_dataset('/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/1979/max/era5-land_global_daily_tmax_197901.nc')

ref.attrs.pop('history')

'Sun Dec 19 20:59:51 2021: cdo -z zip -b i32 mergetime ../era5-land_global_daily_tmax_19790101.nc ../era5-land_global_daily_tmax_19790102.nc ../era5-land_global_daily_tmax_19790103.nc ../era5-land_global_daily_tmax_19790104.nc ../era5-land_global_daily_tmax_19790105.nc ../era5-land_global_daily_tmax_19790106.nc ../era5-land_global_daily_tmax_19790107.nc ../era5-land_global_daily_tmax_19790108.nc ../era5-land_global_daily_tmax_19790109.nc ../era5-land_global_daily_tmax_19790110.nc ../era5-land_global_daily_tmax_19790111.nc ../era5-land_global_daily_tmax_19790112.nc ../era5-land_global_daily_tmax_19790113.nc ../era5-land_global_daily_tmax_19790114.nc ../era5-land_global_daily_tmax_19790115.nc ../era5-land_global_daily_tmax_19790116.nc ../era5-land_global_daily_tmax_19790117.nc ../era5-land_global_daily_tmax_19790118.nc ../era5-land_global_daily_tmax_19790119.nc ../era5-land_global_daily_tmax_19790120.nc ../era5-land_global_daily_tmax_19790121.nc ../era5-land_global_daily_tmax_19790122.nc

In [8]:
var = 'max'


In [17]:
ref.tmax.encoding.copy()

{'zlib': True,
 'shuffle': True,
 'complevel': 1,
 'fletcher32': False,
 'contiguous': False,
 'chunksizes': (1, 1801, 3600),
 'source': '/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/1979/max/era5-land_global_daily_tmax_197901.nc',
 'original_shape': (31, 1801, 3600),
 'dtype': dtype('int32'),
 'missing_value': -9999,
 '_FillValue': -9999,
 'scale_factor': 0.0016127533931795336,
 'add_offset': 264.7066521193972}

In [28]:
def summarise_month_for_var(year, month, var):
    hourly_file = SUBDAILY_TEMPERATURES_FOLDER / f'{year}-{str(month).zfill(2)}_temperature_2m.nc'

    daily_folder = TEMPERATURE_SUMMARY_FOLDER / f'{year}' / var
    daily_folder.mkdir(parents=True, exist_ok=True)

    summary_file = daily_folder / f'era5-land_global_daily_t{var}_{year}{str(month).zfill(2)}.nc'
    print(summary_file)

    hourly_temperatures = xr.open_dataset(hourly_file, chunks=dict(time=-1, longitude=10))
    
    # only for grib version
    # daily_summary = hourly_temperatures.max(dim='step').drop('surface').drop('number')
    daily_summary = hourly_temperatures.resample(time='1D').max()

    daily_summary = daily_summary.rename({'t2m': f't{var}'})

    daily_summary.attrs['frequency'] = 'day'

    encoding = ref.tmax.encoding.copy()
    encoding.pop('source')
    encoding.pop('original_shape')
    encoding.pop('chunksizes')
    encoding.pop('contiguous')

    daily_summary.to_netcdf(summary_file, 
                        encoding={f't{var}':encoding})

In [ ]:
year = 2021
var = 'max'
# for month in range(1, 12+1):
print(year, 1, var)
summarise_month_for_var(year, 1, var)

2021 1 max
/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/2021/max/era5-land_global_daily_tmax_202101.nc


/home/jonathanchambers/Scratch/.conda/envs/science/lib/python3.9/site-packages/dask/utils.py:35: RuntimeWarning: All-NaN slice encountered
  return func(*args, **kwargs)
/home/jonathanchambers/Scratch/.conda/envs/science/lib/python3.9/site-packages/dask/core.py:119: RuntimeWarning: All-NaN slice encountered
  return func(*(_execute_task(a, cache) for a in args))


In [29]:
year = 2021
var = 'max'
for month in range(1, 12+1):
    print(year, month, var)
    summarise_month_for_var(year, month, var)

2021 1 max
/home/jonathanchambers/Shared/Data/weather/era5_land/era5_land_daily_summary/2021/max/era5-land_global_daily_tmax_202101.nc



KeyboardInterrupt



In [ ]:
# add_offset = 272.05038498646843; // double
#   :scale_factor = 0.0014486549927761824; // double

In [7]:
ds = xr.open_dataset('/home/jonathanchambers/Shared/Data/weather/era5_land/hourly_nc/2021-01_temperature_2m.nc')

In [8]:
ds

<xarray.Dataset>
Dimensions:    (longitude: 3600, latitude: 1801, time: 744)
Coordinates:
  * longitude  (longitude) float32 0.0 0.1 0.2 0.3 ... 359.6 359.7 359.8 359.9
  * latitude   (latitude) float32 90.0 89.9 89.8 89.7 ... -89.8 -89.9 -90.0
  * time       (time) datetime64[ns] 2021-01-01 ... 2021-01-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-02-16 15:52:09 GMT by grib_to_netcdf-2.28.0: /home/jon...